This is my capstone project for the Google Data Analytics Certificate. It uses real data from Divvy Bike Share as a fictional company called Cyclistic Bike-Share. The Case study instructions included some R code for us to use which I've marked below as "from script," though I modified some things or changed the order. 

In this scenario I've been asked to compare how Cyclistic members and non-members (casuals) use the bike share. My analysis is to be the first step in determining how to market to casual users to get them to become members. They want me to 1) Describe my data sources; 2) Document my data cleaning; 3) Summarize my Analysis; 4) Provide supporting Visualizations; 5) Provide three recommendations based on my analysis.

**1) Data Sources.** Divvy Bike Share data  (https://divvy-tripdata.s3.amazonaws.com/index.html) is used for the fictional company of the scenario: Cyclistic Bike-Share. I used data from Q2 2019 through Q1 2020 rather than the 2020-21 data because I believe usage of the bike share during the pandemic was exceptional. While some pandemic behaviors will continue, pre-pandemic usage of the service likely better represents how people will behave in the future. If this were a real scenario (and there was access to individual bike user data) I would look at both, since some casual users during the pandemic may want to continue biking. 

As we will see in the cleaning process, data was not recorded consistently in all four quarters, and some of the data collected changed. 

In [ ]:
#install and load packages
install.packages("tidyverse")
install.packages("lubridate")
install.packages("ggplot2")

library(tidyverse)  
library(lubridate)  
library(ggplot2) 


#import datasets. These files have supposed duplicates on Kaggle, but the duplicate had already changed some column names and who knows what else
q2_19 <- read_csv("../input/cyclistic-fictionalization-of-divvy-trip-data/Divvy_Trips_2019_Q2.csv")
q3_19 <- read_csv("../input/cyclistic-fictionalization-of-divvy-trip-data/Divvy_Trips_2019_Q3.csv")
q4_19 <- read_csv("../input/cyclistic-fictionalization-of-divvy-trip-data/Divvy_Trips_2019_Q4.csv")
q1_20 <- read_csv("../input/cyclistic-fictionalization-of-divvy-trip-data/Divvy_Trips_2020_Q1.csv") 
#q1_20 includes the early days of the pandemic. The above code, minus file paths, was from the the script provided by the case study instructions.

**2) Data Cleaning.** First, I standardize the column names to match current practices (ie. the format in 2020 Q1). Then I delete the unneeded columns.

In [ ]:
(q2_19 <- rename(q2_19,
            ride_id = "01 - Rental Details Rental ID",
            rideable_type = "01 - Rental Details Bike ID",
            started_at = "01 - Rental Details Local Start Time",
            ended_at = "01 - Rental Details Local End Time",
            start_station_name = "03 - Rental Start Station Name",
            start_station_id = "03 - Rental Start Station ID",
            end_station_name = "02 - Rental End Station Name",
            end_station_id = "02 - Rental End Station ID",
            member_casual = "User Type"
                ))

(q3_19 <- rename(q3_19,
            ride_id = trip_id,
            rideable_type = bike_id,
            started_at = start_time,
            ended_at = end_time,
            start_station_name = from_station_name,
            start_station_id = from_station_id,
            end_station_name = to_station_name,
            end_station_id = to_station_id,
            member_casual = usertype
                ))

(q4_19 <- rename(q4_19,
            ride_id = trip_id,
            rideable_type = bikeid,
            started_at = start_time,
            ended_at = end_time,
            start_station_name = from_station_name,
            start_station_id = from_station_id,
            end_station_name = to_station_name,
            end_station_id = to_station_id,
            member_casual = usertype #the above commands are from the script
                ))
colnames(q2_19)
q2_19 <- q2_19 %>% 
    select(-c("01 - Rental Details Duration In Seconds Uncapped", "Member Gender", "05 - Member Details Member Birthday Year"))

colnames(q3_19)
q3_19 <- q3_19 %>% 
    select(-c(gender, birthyear, tripduration))

colnames(q4_19)
q4_19 <- q4_19 %>% 
    select(-c(gender, birthyear, tripduration))

colnames(q1_20)
q1_20 <- q1_20 %>% 
    select(-c(start_lat, start_lng, end_lat, end_lng))

Before merging the data I examine it to make sure it is consistent. There are different datatypes in the columns, so I correct this. Finally, I join the data into one data frame and standardize the membership type categories. 

In [ ]:
#examining data to make sure it is consistent. from script
str(q2_19)
str(q3_19)
str(q4_19)
str(q1_20)

#I ultimately don't use these variables, but they should be names (characters) and not numbers to match the Q1 2020 standards. 
#In anycase they should be consistent for joining. These commands from script
q2_19 <- mutate(q2_19, ride_id = as.character(ride_id), 
          rideable_type = as.character(rideable_type))

q3_19 <- mutate(q3_19, ride_id = as.character(ride_id), 
          rideable_type = as.character(rideable_type))

q4_19 <- mutate(q4_19, ride_id = as.character(ride_id), 
          rideable_type = as.character(rideable_type))

#time isn't formated properly for q3_19. Also this quarter doesn't have seconds, which is irrelevant for our purposes. 
#I scrub seconds from the rest of the data below so it doesn't mess with my scatterplot
q3_19$started_at <- mdy_hm(q3_19$started_at)
q3_19$ended_at <- mdy_hm(q3_19$ended_at)

#q1_20 alreay has the correct types

#merge four quarters into one dataframe. from script
full_year <- bind_rows(q2_19, q3_19, q4_19, q1_20)

#use uniform coding for member/casual. from script
full_year <- full_year %>% mutate(member_casual = recode(member_casual,
                                "Subscriber" = "member",
                                "Customer" = "casual"))

**3) Summary of my analysis.** The data has been anonymized, and so there's no easy way to track the behavior of individual riders (most interesting would be whether casual users repeatedly rent, or if they are making a similar trips repeatedly). We can, though, look at when the different types of riders ride, how long they tend to ride, and which stations are most popular.

In [ ]:
#breaking down the datetime data so we can analyse it
full_year$time_of_day <- hms::as_hms(round_date(full_year$started_at, unit = "min")) #because one data set didn't have seconds, 
#my scatterplot was giving me a shadow line from all the rides at 0secs. We don't need this data (unless we were trying 
#to determine waittimes/lines for renting).
full_year$date <- as.Date(full_year$started_at) #time formating from script
full_year$month <- format(as.Date(full_year$date), "%m")
full_year$day <- format(as.Date(full_year$date), "%d")
full_year$year <- format(as.Date(full_year$date), "%y")
full_year$ride_length <- difftime(full_year$ended_at, full_year$started_at) #this is to determine how long people rode. There are some absurdly long times in here. There's noone to ask at this fictional company about it. Do people rent bikes like cars for a week, or do they loose them or forget to return them?

#convert ride_length to a number so it can be calculated. from script
full_year$ride_length <-as.numeric(as.character(full_year$ride_length))

#create column for most popular trips
full_year$popular_trips <- paste(full_year$start_station_name, " to ", full_year$end_station_name)

#calculate the day of week. from script
full_year$day_of_week <- format(as.Date(full_year$date), "%A")

#remove where bikes came out of circulation. The negative ride lengths are removed since it is not an issue of rolling over 
#midnight (since entire dates were compared not just time of day), so there must be a problem with the underlying data. 
#I created a new dataframe where data is removed to preserve to original data with my previous calculations. #from script
clean_fyear <- full_year[!(full_year$start_station_name == "HQ QR" | full_year$ride_length < 0),]


The new columns prepared above now allow us to do some analysis.

In [ ]:
#Now we perform some calculations to begin to see how members and casuals differ. #summary and aggregations from script 
summary(clean_fyear$ride_length)

aggregate(clean_fyear$ride_length ~ clean_fyear$member_casual, FUN = mean)
aggregate(clean_fyear$ride_length ~ clean_fyear$member_casual, FUN = median)
aggregate(clean_fyear$ride_length ~ clean_fyear$member_casual, FUN = max) 
aggregate(clean_fyear$ride_length ~ clean_fyear$member_casual, FUN = min) 

short_rides <- filter(clean_fyear, ride_length < 180 & start_station_name == end_station_name)
count(short_rides, member_casual) #members turn their bikes in quickly to the same station 5x more than casuals.
count(clean_fyear, member_casual) #to see total number of rides. members rides, though are a bit less than 5x casual rides. So Members are more likely to return a bike right away.

The high max ride length values seem to me to be where something went wrong. I don't see why anyone would intentionally keep the bike for more than 48 hours. Why not return it and get another? Are they riding to where there are no stations to spend the night, then coming back? If this were real I’d definitely ask the stakeholders about this. I would want to cut off all unusual behavior before doing an analysis, but I lack data to determine an appropriate cutoff.

On the other extreme, checking a bike out for less than 3 minutes is likely people who found a mechanical problem, or accidentally returned it, or changed their minds about riding. If we had individual user data we could see if they again checked a bike out shortly after or not. For casual users, this could be a data point relevant for people on the fence about trying the service. I’m leaving these extreme values in my data since I don’t know enough about them to determine appropriate cutoff points. Will someone try to get a bike to work for 4 minutes before returning it? Are these people who are changing their mind due to social reasons? Do they see their bus arriving? Is it rainy or too cold? In any case it’s tangential to my main question. 

The difference between the membership type for mean and median show that the casual users go on far longer rides. My hunch is that they are doing joyrides, while the members are riding the last leg of their commute to work. If this is the case, we should see this in terms of the day they are riding.

In [ ]:
#use words for day of week, instead of numbers. the below is from script
clean_fyear$day_of_week <-ordered(clean_fyear$day_of_week, levels = c("Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"))

#does ride length vary by day? 
aggregate(clean_fyear$ride_length ~ clean_fyear$member_casual + clean_fyear$day_of_week, FUN = mean)

#lets see this more cleanly
clean_fyear %>% mutate(weekday = wday(started_at, label = TRUE)) %>%
      group_by(member_casual, weekday) %>%
      summarise(number_of_rides = n(), average_duration = mean(ride_length)) %>%
      arrange(member_casual, weekday)


**4) Provide supporting visualizations.** As we can see below, average ride length does differ for both groups for the day of the week. Casual users tend to go on their longest rides mid-week, and members tend to go on their longest rides on weekends. When each group does the most rides, though, is informative: casual riders do more rides on the weekend while members do more rides on weekdays. This is what we would expect if members are commuters and casual users are tourists/recreational.

I made the following visualizations to show the differences in bike usage.

In [ ]:
#here we compare members and casuals for the number of rides on each day of the week. from script, though I added labels 
clean_fyear %>% mutate(weekday = wday(started_at, label = TRUE)) %>%
      group_by(member_casual, weekday) %>%
      summarise(number_of_rides = n(), average_duration = mean(ride_length)) %>%
      arrange(member_casual, weekday) %>%
      ggplot(aes(x = weekday, y = number_of_rides, fill = member_casual)) +
        geom_col(position = "dodge") +
        labs(title = "Traffic by Day of Week", x = NULL, y = "Number of Rides", fill = "Membership Status") +
        scale_y_continuous(labels = scales::comma)

#average ride duration by day. from script, I added lables and displayed in mins
clean_fyear %>% mutate(weekday = wday(started_at, label = TRUE)) %>%
      group_by(member_casual, weekday) %>%
      summarise(number_of_rides = n(), average_duration = mean(ride_length)) %>%
      arrange(member_casual, weekday) %>%
      ggplot(aes(x = weekday, y = average_duration/60, fill = member_casual)) +
        geom_col(position = "dodge") +
        labs(title = "Ride Length per Week", x = NULL, y = "Average Duration in Mins", fill = "Membership Status") +
        scale_y_continuous(labels = scales::comma)

#traffic by month. Go figure, only the real die-hards want to ride a bike in Chicago in the winter. 
#Comparing the exact dates to weather data would show us who's willing to ride in the snow. 
#Plowing schedule data might help determine which stations are worth keeping open in the snowy season and which will be buried.
#Casual rides in the winter are a smaller propotion of total rides, than is the case for Members.
clean_fyear %>% 
      group_by(member_casual, month) %>%
      summarise(number_of_rides = n()) %>%
      arrange(member_casual, month) %>%
      ggplot(aes(x = month, y = number_of_rides, fill = member_casual)) +
        geom_col(position = "dodge") +
        labs(title = "Traffic per Month", x = "Month", y = "Number of Rides", fill = "Membership Status") +
        scale_y_continuous(labels = scales::comma)

#traffic by time of day. Here we see clearly that most member ussage is rush hour commuting, and most casual ussage is midday joy rides.
clean_fyear %>% 
      group_by(member_casual, time_of_day) %>%
      summarise(number_of_rides = n()) %>%
      arrange(member_casual, time_of_day) %>%
      ggplot(aes(x = time_of_day, y = number_of_rides, color = member_casual)) +
        geom_point() +
        labs(title = "Traffic Time of Day", x = "Time", y = "Number of Rides", fill = "Membership Status") +
        scale_x_time(breaks = hm("06:00", "12:00", "18:00"))  + 
        scale_y_continuous(labels = scales::comma)


        

The casual = tourist, member = commuter hypothosis is further supported if you look at the most popular stations for each group below.

In [ ]:
#all stations ranked by traffic amount, grouped by membership status
popular_station <- clean_fyear %>% group_by(start_station_name, member_casual) %>%
      summarise(number_of_rides = n()) %>%
      arrange(member_casual, -number_of_rides)
      
popular_endpoint  <- clean_fyear %>% group_by(end_station_name, member_casual) %>%
      summarise(number_of_rides = n()) %>%
      arrange(member_casual, -number_of_rides)
         
popular_rides <-  clean_fyear %>% group_by(popular_trips, member_casual) %>%
      summarise(number_of_rides = n()) %>%
      arrange(member_casual, -number_of_rides)  

count(clean_fyear, member_casual)

The top 10 starting stations for casual riders are all along the lakefront bike trail and are near tourist destinations. Rides starting from these 10 stations constitute over 25% of total rides casuals are making. 

The top 3 starting stations for members are right outside the Ogilvie Transportation Center, another of the top 10 is outside Union Station, and two more are near Millennium Park Station. The rest are all around busy commercial areas like Daily Plaza and LaSalle St. The top 10 ending stations for members are all stations in the top 12 starting stations, though in different order. This suggests many are commuters going one way in the morning and the opposite at night. The rides from these 10 starting stations constitute over 11% of total rides. We could no doubt dig deeper to show even more regularity of starting/stopping station popularity.


** 5) Provide three recommendations based on my analysis.** 
The question for this study was: how do members and casuals use Cyclistic bikes differently? This comes in the context of the marketing team’s desire to launch a campaign to convince casual users to sign on for a membership. My three recommendations are:
    First, for the most part, members are commuters and casual users are tourists. There may be a segment of commuter who will occasionally rent a bike to commute. A close examination of casual rides on weekdays, during rush hour, near train stations and commercial areas may reveal this target user. Further analysis should also determine whether there are enough of such users to make the campaign worthwhile.
    Second, the casual rider tends to ride mid-day or on weekends on the Lakefront trail. Without individual data we can't tell if the same people are doing this regularly. Perhaps some of these riders enjoy this enough to become members, though you should determine how or why this would appeal to them more than buying a bike.
    Third, given that members mostly behave like commuters, and casual users mostly behave like tourists, I would not recommend putting too many resources into trying to get casuals to become members (if you do have data on specific users, this might be worthwhile). Rather, I would recommend focusing on convincing more tourists to act like casual users and convince more commuters they should become members.


As part of this project I prepared a few visuals on Tableau. They can be accessed here: https://public.tableau.com/shared/S5CMBBYQM?:display_count=n&:origin=viz_share_link